# Optimizing Ski Students and Teachers Pickup

**Develop an algorithm to solve the problem outlined below.**

Your code must solve all instances provided in the [GitHup repository of the course](https://github.com/Daddeee/FRO_Labs_22-23/tree/master/big-project) in **less than 30 minutes** of execution time.

The last cell of your notebook should print the results of your algorithm as if it was a csv file with 3 columns:

*   "Id": the ID of the instance (1, 2, 3, ..., 8)
*   "Obj": the objective function value obtained
*   "Time": the execution time in seconds.

**This notebook already contains the skeleton of code to produce the correct output**. 
You only have to include your solution in the "solve" function below.

Once you have completed your code, you can upload it to the [big-project WeBeep assignment](https://webeep.polimi.it/mod/assign/view.php?id=171992).

For instances 1, 2, 3 and 4, we provide you with the optimal objective function value ([here on GitHub](https://github.com/Daddeee/FRO_Labs_22-23/tree/master/big-project/solutions)). You can compare the output of your algorithm with these results to understand how well you are performing.

## General info
*   groups of max 3 students
*   deadline at 09/06/2023 23:59 CET
*   NO pre-coded libraries or algorithms.

## Evaluation
*   4 lab points if you deliver something that works.
*   10 points based on the quality of your solutions, measured in gap w.r.t. optimal solutions and execution times.
*   Execution times will be re-examined on a random basis.

## Problem

A ski school provides transportation for its students.

The ski school operates a fleet of $k$ buses, each capable of transporting a maximum of $C$ students. Based on the enrollments for the upcoming winter, the school expects to pick up its students from a set of $n$ neighboring towns. Each town has $d_i$ students and must be visited exactly once by one of the buses.

The school has a total of $k$ teachers (one per bus). Each day, some teachers (not necessarily all $k$ of them) will drive a bus to pickup all students. When an instructor is not driving a bus, he must be picked up by one of the other buses. The ski school pays a fixed daily fee for each instructor that drives a bus, equivalent to the distance between his hometown and the school.

To ensure efficient transportation, a bus visiting a town must pick up all of its students, and the total number of students picked up by each bus must not exceed its capacity. Additionally, it is mandatory for each bus to start and end its route at the ski school.

Your goal is to plan the pickup routes of the ski school with the goal of minimizing the total distance travelled by buses and the fixed cost of each teacher that is driving a bus.


## Instance

Each instance is a ".json" file containing all information needed to solve the problem. Its fields are:

*   *town_coordinates*: the coordinates $(x,y) \in [0,100]^2$ of each town,
*   *depot_coordinates*: the coordinates $(x,y) \in [0,100]^2$ of the depot,
*   *teacher_coordinates*: the coordinates $(x,y) \in [0,100]^2$ of the hometown of each teacher,
*   *students_per_town*: the number of students waiting in each town.
*   *bus_capacity*: the maximum number of people that can travel on each bus.


In [1]:
# Run this cell to download all instances from the GitHub repository.
# It also downloads results for instances 1, 2, 3, and 4.

# Clean files if present
!rm instance_1.json
!rm result_1.txt
!rm instance_2.json
!rm result_2.txt
!rm instance_3.json
!rm result_3.txt
!rm instance_4.json
!rm result_4.txt
!rm instance_5.json
!rm instance_6.json
!rm instance_7.json
!rm instance_8.json

# Download directly from Github
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_1.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_2.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_3.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_4.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_5.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_6.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_7.json
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_8.json

!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/results/result_1.txt
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/results/result_2.txt
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/results/result_3.txt
!wget https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/results/result_4.txt

rm: cannot remove 'instance_1.json': No such file or directory
rm: cannot remove 'result_1.txt': No such file or directory
rm: cannot remove 'instance_2.json': No such file or directory
rm: cannot remove 'result_2.txt': No such file or directory
rm: cannot remove 'instance_3.json': No such file or directory
rm: cannot remove 'result_3.txt': No such file or directory
rm: cannot remove 'instance_4.json': No such file or directory
rm: cannot remove 'result_4.txt': No such file or directory
rm: cannot remove 'instance_5.json': No such file or directory
rm: cannot remove 'instance_6.json': No such file or directory
rm: cannot remove 'instance_7.json': No such file or directory
rm: cannot remove 'instance_8.json': No such file or directory
--2023-06-07 16:24:58--  https://raw.githubusercontent.com/Daddeee/FRO_Labs_22-23/master/big-project/instances/instance_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connect

In [2]:
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 11.8 MB/s eta 0:00:00


In [11]:
import numpy as np
import mip
import json
import math
import datetime

# METODI:

# get_cycles
def get_cycles(V, A, x):
  graph = [[] for i in V]

  for (i,j) in A:
    if x[i,j].x > 0.5:
      graph[i].append(j)

  cycles = []
  color = [0 for i in V]
  par = [-1 for i in V]

  for i in V:
    if par[i] == -1:
      dfs_cycle(graph, cycles, i, -1, color, par)

  for u in V:
    adj = [v for v in graph[u] if v > u]
    for v in adj:
      if u in graph[v]:
        cycles.append([u,v])

  return [c for c in cycles if len(c) < len(V) and len(c) > 0 and not 0 in c]

# dfs_cycles
def dfs_cycle(graph, cycles, u, p, color, par):
    if color[u] == 2:
        return

    if color[u] == 1:
        v = []
        cur = p
        v.append(cur)
        while cur != u:
            cur = par[cur]
            v.append(cur)
        cycles.append(v)
        return
 
    par[u] = p
    color[u] = 1
 
    for v in graph[u]:  
        if v == par[u]:
            continue
        dfs_cycle(graph, cycles, v, u, color, par)
    color[u] = 2

# solutore_singolo
def solutore_singolo(depot,towns,teachers,student_numbers,bus_capacity):

  def r(S):
      student_numbers_completo = student_numbers[:]
      student_numbers_completo.insert(0,0)
      sum_d = sum([student_numbers_completo[i] for i in S])
      return int(np.ceil(sum_d/bus_capacity))

  C = bus_capacity 
  k = len(teachers) 
  K = range(k)  
  n = len(towns)+1
  V = range(n) 
  V0 = V[1:] 
  d = [0] + student_numbers
  count=0
  A = [(i,j) for i in V for j in V] 

  point = np.array(depot+towns)
 
  depot_towns_completo = depot+towns[:]
  coordinates_temp = {i: (depot_towns_completo[i][0], depot_towns_completo[i][1]) for i in V}
  coordinates_temp = point

  # distanza tra ogni città
  c = np.array([[math.sqrt(np.sum((point[i] - point[j])**2)) for i in V] for j in V])
  
  m = mip.Model()

  # variabili
  x = {(i,j): m.add_var(var_type=mip.BINARY) for (i,j) in A}

  # vincoli
  for j in V:
    num_outgoing_arcs = num_ingoing_arcs = 1
    if j == 0:
      num_outgoing_arcs = num_ingoing_arcs = k

    m += mip.xsum(x[i,j] for i in V) == num_ingoing_arcs
    m += mip.xsum(x[j,i] for i in V) == num_outgoing_arcs
  
  # vincolo di eliminazione autoanelli
  for i in V:
    m += x[i,i]==0
  
  # funzione obiettivo
  m.objective = mip.minimize(mip.xsum(c[i,j] * x[i,j] for i in V for j in V))
  m.optimize()
  
  # eliminazione sottocicli
  # mediante variabile count si tiene conto delle chiamate a m.optimize(), se troppo elevate non conveniente

  cycles = get_cycles(V, A, x)
  while len(cycles) > 0 and count<10:
    for cycle in cycles:
      cycle_edges = [x[i,j] for (i,j) in A if i in cycle and j in cycle]
      m += mip.xsum(cycle_edges) <= len(cycle) - r(cycle)

    m.optimize()
    count = count+1

    cycles = get_cycles(V, A, x)

    if len(cycles) == 0:
      for j in V:
        if j == 0:
          continue
        
        if x[0,j].x > 0.1:
          route = [0,j]
          prev = route[-1]
          while prev != 0:
            succ = -1
            for k in V:
              if k == prev:
                continue
              if x[prev,k].x > 0.1:
                succ = k
                break
            route.append(succ)
            if succ == -1:
              raise ValueError("Something went wrong with route={}".format(route))
            prev = route[-1]

          sum_d = sum([d[v] for v in route])
          if sum_d > C:
            cycles.append(route[1:-1])

    soluzione_ottima = m.objective_value

  if count > 9:
    from itertools import chain, combinations
    powerset = list(chain.from_iterable(combinations(V0, r) for r in V0))
    
    # dopo un certo numero di chiamate della m.optimize() si reintroducono in blocco tutti i vincoli sui sottocicli
    for S in powerset:
      if len(S) >= 1:
        m += mip.xsum( mip.xsum(x[i,j] for j in S) for i in S) <= len(S)-r(S)
    m.optimize()
    soluzione_ottima = m.objective_value


  if soluzione_ottima != None:
    
    # si aggiunge alla soluzione trovata il costo dei teachers che guidano
    teachers_fixed_daily_fee_np  = np.array(teachers)
    
    for t in teachers_fixed_daily_fee_np:     
      soluzione_ottima += math.sqrt(np.sum((t-point[0])**2))

  else:
    soluzione_ottima = 999999999

  return soluzione_ottima

In [12]:
# Reads a .json instance and returns it in a dictionary
def load_instance(filename):
  with open(filename, 'r') as f:
    data = json.load(f)
  return data

# Reads a .txt result and returns it
def load_result(filename):
  try:
    with open(filename, 'r') as f:
      return float(f.read())
  except FileNotFoundError:
    return None

def solve(instance):

  k = len(instance["teacher_coordinates"])
  K = range(k)  

  # calcolo il minimo numero di bus necessari
  min_bus = math.ceil(sum(instance["students_per_town"])/instance["bus_capacity"])

  # powerset bus
  from itertools import chain, combinations 
  powerset_bus_no_teacher = list(chain.from_iterable(combinations(K, r) for r in range(k-min_bus+1)))
  
  risultati = []

  for bus_no_teacher in powerset_bus_no_teacher:

    town_coordinates_temp = instance["town_coordinates"][:]
    teacher_coordinates_temp = instance["teacher_coordinates"][:]
    students_per_town_temp = instance["students_per_town"][:]
    
    # sposto i teachers che non guidano negli studenti 
    for bus in bus_no_teacher: 
      town_coordinates_temp.append(instance["teacher_coordinates"][bus])
      students_per_town_temp.append(1)

    for bus in bus_no_teacher: 
      del teacher_coordinates_temp[bus]

    risultato_temp = solutore_singolo(instance["depot_coordinates"], town_coordinates_temp, teacher_coordinates_temp, students_per_town_temp, instance["bus_capacity"])
    risultati.append(risultato_temp)
    
  return min(risultati)

insts = [1, 2, 3, 4, 5, 6, 7, 8]
outputs = []
times = []

for i in insts:
  instance_name = "instance_{}.json".format(i)
  results_name = "result_{}.json".format(i)
  inst = load_instance(instance_name)

  # start and end are used to measure execution time
  start = datetime.datetime.now()
  
  obj = solve(inst)
  
  end = datetime.datetime.now()
  time = (end - start).total_seconds()

  print("[{}] Found solution with obj: {}".format(i, obj))
  print("[{}] Elapsed time: {} s".format(i, time))

  res = load_result(results_name)
  if res is not None:
    gap = 100 * (obj - res) / res
    print("[{}] Gap: {}".format(i, gap))

  outputs.append(obj)
  times.append(time)

print("Id,Obj,Time")
for i in range(len(insts)):
  print(insts[i], outputs[i], times[i])

[1] Found solution with obj: 458.27729249529904
[1] Elapsed time: 0.049882 s
[2] Found solution with obj: 712.5038409416293
[2] Elapsed time: 0.178783 s
[3] Found solution with obj: 414.8740205020167
[3] Elapsed time: 0.056089 s
[4] Found solution with obj: 442.3864167820078
[4] Elapsed time: 0.321141 s
[5] Found solution with obj: 536.038540063322
[5] Elapsed time: 0.133572 s
[6] Found solution with obj: 718.4242962117113
[6] Elapsed time: 0.42633 s
[7] Found solution with obj: 476.0452590408317
[7] Elapsed time: 0.087978 s
[8] Found solution with obj: 690.5789320059423
[8] Elapsed time: 6.457932 s
Id,Obj,Time
1 458.27729249529904 0.049882
2 712.5038409416293 0.178783
3 414.8740205020167 0.056089
4 442.3864167820078 0.321141
5 536.038540063322 0.133572
6 718.4242962117113 0.42633
7 476.0452590408317 0.087978
8 690.5789320059423 6.457932
